In [1]:
!kubectl create namespace seldon
!kubectl config set-context $(kubectl config current-context) --namespace=seldon
import json

Error from server (AlreadyExists): namespaces "seldon" already exists
Context "gke_airy-berm-306315_us-central1-c_ambassador-1" modified.


## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the
tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the
TFServing prepackaged serve with the Seldon Protocol, and a second one which
shows how you can deploy it using the tensorlfow protocol (so you can send
requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Seldon protocol.

In [3]:
%%writefile servers/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores     
    name: default
    replicas: 1

Writing servers/mnist_rest.yaml


In [4]:
!kubectl apply -f servers/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [5]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "tfserving-default-0-mnist-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "tfserving-default-0-mnist-model" successfully rolled out


In [6]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="tfserving", namespace="seldon")

#### REST Request

In [7]:
r = sc.predict(gateway="ambassador", transport="rest", shape=(1, 784))
print(r)
assert r.success == True

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.8611661987296505
    values: 0.5788173027055631
    values: 0.6974848873863065
    values: 0.645703614361863
    values: 0.8700594087953364
    values: 0.9409208546095615
    values: 0.4531646939530869
    values: 0.13121329925209568
    values: 0.30407474621215713
    values: 0.8111954533710641
    values: 0.9998606271397951
    values: 0.46916835964693127
    values: 0.7926850602531341
    values: 0.021482736483198317
    values: 0.18492752037436122
    values: 0.46669412220244144
    values: 0.5857758465930641
    values: 0.4580212177765539
    values: 0.18363934545002947
    values: 0.9788130196441382
    values: 0.09174777705003645
    values: 0.949983004836309
    values: 0.6227434601076408
    values: 0.7948614797166639
    values: 0.46104736248750045
    values: 0.2965483091314751
    values: 0.7444575356578712
    values: 0.20293644246682885
    values: 0.7987040936437609
    va

#### gRPC Request

In [8]:
r = sc.predict(gateway="ambassador", transport="grpc", shape=(1, 784))
print(r)
assert r.success == True

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 784], 'values': [0.6529444963059101, 0.9058488793813774, 0.8238582317348911, 0.6261504201777014, 0.33379659122270144, 0.3113620541370319, 0.23954970936905728, 0.74518245692369, 0.5611309795448415, 0.5608160661572328, 0.1977220961921028, 0.09633522187296484, 0.7817307108854458, 0.4782723943815763, 0.3489850377571766, 0.3404474214393961, 0.06186088315132987, 0.4647760103913492, 0.4090389525817646, 0.30034206916222805, 0.2202318020059476, 0.049373448048568025, 0.9395183036214291, 0.47106046569593907, 0.7839736024268041, 0.41739293948294665, 0.5334564624440442, 0.06475003943623414, 0.05089704873676748, 0.9586865929478624, 0.5825699886046509, 0.33244232204460844, 0.1286961198701001, 0.5627753436681688, 0.45893443114214005, 0.9537097073870228, 0.7823862959024463, 0.38607195663524674, 0.4495125408927264, 0.6788388401521405, 0.03133004937885098, 0.39678102598037457, 0.3343603030481431, 0.18536841583106833, 0.061302760

And delete the model we deployed

In [9]:
!kubectl delete -f servers/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


### Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Tensorflow protocol.

In [11]:
%%writefile servers/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Writing servers/halfplustwo_rest.yaml


In [15]:
!kubectl apply -f servers/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io/hpt created


In [16]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "hpt-default-0-halfplustwo" rollout to finish: 0 of 1 updated replicas are available...
deployment "hpt-default-0-halfplustwo" successfully rolled out


In [18]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

{'predictions': [2.5, 3.0, 4.5]}


In [19]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:hpt -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
!kubectl delete -f servers/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io "hpt" deleted
